<a href="https://colab.research.google.com/github/Stray-Weights/Dlthon_01_Motocycle_Night_Ride/blob/main/final_work/3_%EA%B8%B0%EB%8A%A5%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

===============================================================================
이 파일은 야간 주행 안전 지수 시스템의 전체 파이프라인을 구현합니다:
  1. 박항아: 세그멘테이션 모델 학습 및 예측
  2. 박무림: 예측 결과 분석 및 최종 리포트 생성
===============================================================================
"""

# ============================================================================
# [박항아 파일] Step 0 - 라이브러리 임포트
# ============================================================================

In [ ]:
import cv2
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from torchvision.transforms import InterpolationMode
from dotenv import load_dotenv

# 추가 임포트 (박무림 파일용)
from tqdm.auto import tqdm
import pandas as pd
from dataclasses import dataclass
from sklearn.model_selection import train_test_split

# Device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"현재 사용 중인 장치: {device}")


현재 사용 중인 장치: cpu


# ============================================================================
# [박항아 파일] PathManager 클래스 선언
# ============================================================================

In [ ]:
class PathManager:
    """환경 및 모듈 간 규격 클래스"""
    """상황에 맞게 경로와 규격을 수정 후 사용할 것"""

    def __init__(self, project_path: str | Path | None = None, data_sub_path: str = "data", default_size: int | None = None):
        # 1. 루트 경로 결정 (제공된 경로가 없으면 자동 탐색)
        if project_path:
            self.root_path = Path(project_path)
            print(f"📁 설정된 프로젝트 경로 사용: {self.root_path}")
        else:
            self.root_path = self._find_root()
            print(f"🔍 자동 탐색된 프로젝트 경로: {self.root_path}")

        # 2. .env 로드
        self.dotenv_path = self.root_path / ".env"
        if self.dotenv_path.exists():
            load_dotenv(self.dotenv_path)
            print("✅ .env 환경 변수 로드 완료")
        else:
            print("⚠️  .env 파일을 찾을 수 없습니다. 기본 설정을 사용합니다.")

        load_dotenv(self.dotenv_path)

        # 3. 디렉토리 경로 설정
        self._datas_dir = self.root_path / data_sub_path
        self._images_dir = self._datas_dir / 'images'
        self._models_dir = self.root_path / 'models'

        # 4. 자동 폴더 생성 로직
        self._make_dirs()

        self.default_size = default_size
        self.image_name = None
        self.model_name = None

    def _find_root(self, marker: str = ".env") -> Path:
        """프로젝트 최상단 경로를 찾는 함수"""
        current_cwd = Path().cwd().resolve()

        if (current_cwd / marker).exists():
            return current_cwd

        for parent in current_cwd.parents:
            if (parent / marker).exists():
                return parent

        return current_cwd  # 기본값

    def _make_dirs(self):
        """필요한 디렉토리가 없으면 생성합니다."""
        target_dirs = [self._images_dir, self._models_dir, self._datas_dir]
        for path in target_dirs:
            path.mkdir(parents=True, exist_ok=True)
            print(f"준비 완료: {path}")

    # --- 경로 획득 메서드 ---
    def get_image_path(self, image_name: str) -> Path:
        """이미지 주소 획득"""
        self.image_name = image_name
        return self._images_dir / image_name

    def get_model_path(self, model_name: str) -> Path:
        """모델 주소 획득"""
        self.model_name = model_name
        return self._models_dir / model_name

    def get_data_path(self, data_name: str) -> Path:
        """데이터 주소 획득"""
        self.data_name = data_name
        return self._datas_dir / data_name

# ============================================================================
# [박무림 파일] Section 1: 경로 관리 클래스 (PathManager 통합)
# ============================================================================

In [ ]:
class ResultPathManager:
    """분석 결과 저장 경로를 관리하는 클래스"""

    def __init__(self, base_path=None):
        if base_path is None:
            if os.path.exists('/content/drive'):
                base_path = '/content/drive/MyDrive/murim_results'
            else:
                base_path = './results'

        self.base_path = Path(base_path)
        self.base_path.mkdir(parents=True, exist_ok=True)
        print(f"✅ ResultPathManager 초기화: {self.base_path}")

    def get_data_path(self, filename):
        """데이터 저장 경로 반환"""
        path = self.base_path / filename
        path.parent.mkdir(parents=True, exist_ok=True)
        return str(path)

    def get_json_path(self):
        """Result.json 경로"""
        return self.get_data_path("Result.json")

    def get_analysis_path(self):
        """분석 결과 CSV 경로"""
        return self.get_data_path("analysis_results.csv")


# ============================================================================
# [박무림 파일] Section 2: 예측 결과를 COCO JSON으로 저장
# ============================================================================

In [ ]:
def save_predictions_to_coco(model, test_loader, device, class_map, path_manager):
    """
    ========================================================================
    예측 결과를 COCO 형식의 JSON으로 변환 및 저장
    ========================================================================

    역할:
        - 세그멘테이션 모델의 출력을 COCO 형식의 JSON으로 변환
        - 각 객체의 폴리곤, 바운딩박스, 면적 정보 포함

    입력:
        model: PyTorch 세그멘테이션 모델 (output['out'] shape: [B, C, H, W])
        test_loader: DataLoader (이미지, _ 튜플 반환)
        device: 'cuda' 또는 'cpu'
        class_map: 클래스 매핑 {"Undrivable": 0, "Drivable": 1, ...}
        path_manager: PathManager 또는 ResultPathManager 인스턴스

    출력:
        - Result.json 파일 생성
        - 딕셔너리 반환 (images, annotations, categories)
    ========================================================================
    """
    model.eval()

    # 1. COCO JSON 기본 구조
    results = {
        "images": [],
        "annotations": [],
        "categories": [{"id": v, "name": k} for k, v in class_map.items()]
    }

    ann_id = 1
    image_id_count = 1

    print("\n⏳ 예측 결과를 COCO JSON으로 변환 중...")

    with torch.no_grad():
        for images, _ in tqdm(test_loader, desc="Processing images"):
            images = images.to(device)

            # 모델 예측
            outputs = model(images)['out']

            # 각 픽셀에서 확률이 가장 높은 클래스 선택
            preds = torch.argmax(outputs, dim=1).cpu().numpy()  # [Batch, H, W]

            for batch_idx in range(len(preds)):
                file_name = f"result_{image_id_count:06d}.png"

                # 2. 이미지 정보 추가
                results["images"].append({
                    "id": image_id_count,
                    "file_name": file_name,
                    "width": 528,
                    "height": 528
                })

                # 3. 각 클래스별로 어노테이션 생성
                pred_mask = preds[batch_idx]  # [528, 528]

                for class_name, class_idx in class_map.items():
                    if class_idx == 0:
                        continue  # 배경(Undrivable) 제외

                    # 해당 클래스의 이진 마스크 생성
                    class_mask = (pred_mask == class_idx).astype(np.uint8)

                    # 외곽선(Contour) 찾기
                    contours, _ = cv2.findContours(
                        class_mask,
                        cv2.RETR_EXTERNAL,
                        cv2.CHAIN_APPROX_SIMPLE
                    )

                    for contour in contours:
                        # 최소 3개의 점 필요
                        if len(contour) < 3:
                            continue

                        # COCO 폴리곤 형식: [x1, y1, x2, y2, ...]
                        segmentation = contour.flatten().tolist()

                        # 바운딩박스: [x, y, width, height]
                        x, y, w, h = cv2.boundingRect(contour)
                        area = float(cv2.contourArea(contour))

                        # 너무 작은 노이즈 제거
                        if area < 10:
                            continue

                        # 어노테이션 추가
                        results["annotations"].append({
                            "id": ann_id,
                            "image_id": image_id_count,
                            "category_id": class_idx,
                            "segmentation": [segmentation],
                            "area": area,
                            "bbox": [x, y, w, h],
                            "iscrowd": 0
                        })
                        ann_id += 1

                image_id_count += 1

    # 4. JSON 파일 저장
    save_path = path_manager.get_json_path()
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    # 5. 결과 출력
    print(f"\n✅ COCO JSON 저장 완료")
    print(f"   경로: {save_path}")
    print(f"   📊 통계:")
    print(f"      - 이미지 수: {len(results['images'])}")
    print(f"      - 어노테이션 수: {len(results['annotations'])}")
    print(f"      - 카테고리 수: {len(results['categories'])}")

    return results



# ============================================================================
# [박무림 파일] Section 3: 최종 통합 분석 클래스
# ============================================================================

In [ ]:
class MurimFinalIntegration:
    """박무림이 개발한 지능형 자율주행 분석 엔진"""

    def __init__(self):
        self.prev_distance = None  # 시계열 분석용 메모리
        # 물리 파라미터
        self.camera_h = 1.2
        self.focal_l = 1200

    def process_step_34_logic(self, annotations, img_info):
        """
        [기존 섹션 3, 4의 기능을 모두 포함]
        팀장님의 JSON 데이터를 받아 거리 계산 및 위험 판단 수행
        """
        img_h = img_info['height']

        # [Task 2] 고유 ID 식별
        ID_RIDER = 1329681      # Rider
        ID_MOVEABLE = 1323884   # Moveable (차량)
        ID_ROAD = 1323881       # Road

        # 객체 필터링
        rider_anns = [a for a in annotations if a['category_id'] == ID_RIDER]
        moveable_anns = [a for a in annotations if a['category_id'] == ID_MOVEABLE]

        # [Task 5] 도로 혼잡도 (차량 대수 확인)
        moveable_count = len(moveable_anns)
        congestion = "혼잡" if moveable_count >= 3 else "원활"

        # 결과 저장 기본 구조
        report = {"pixel_dist": 0, "real_dist": 0, "congestion": congestion, "status": "SAFE", "count": moveable_count}

        if rider_anns and moveable_anns:
            # 1. [Task 3] 픽셀 거리 계산 (Rider vs Moveable)
            rx, ry, rw, rh = rider_anns[0]['bbox']
            mx, my, mw, mh = moveable_anns[0]['bbox']

            # 하단 중앙점 기준 픽셀 간격 계산
            pix_dist = np.sqrt(((rx+rw/2)-(mx+mw/2))**2 + ((ry+rh)-(my+mh))**2)
            report["pixel_dist"] = round(pix_dist, 2)

            # 2. [Task 4] 실제 미터(m) 변환 (기존 섹션 3의 핵심 물리 공식)
            v_horizon = img_h * 0.5
            # d = (H * f) / (y - v_horizon) -> 핵심 거리 공식
            real_dist = (self.camera_h * self.focal_l) / (my + mh - v_horizon) if (my + mh - v_horizon) > 0 else 50.0
            report["real_dist"] = round(real_dist, 2)

            # 3. [Task 6] 시계열 충돌 경고
            if self.prev_distance is not None:
                distance_change = self.prev_distance - real_dist
                if distance_change > 0.5:  # 0.5m 이상 접근
                    report["status"] = "⚠️  WARNING"
                if distance_change > 2.0:  # 2m 이상 급접근
                    report["status"] = "🚨 CRITICAL"

            self.prev_distance = real_dist

        return report


# ============================================================================
# [박무림 파일] Section 4: 결과 분석 함수
# ============================================================================

In [ ]:
def process_team_result_json(result_json=None, result_json_path=None, path_manager=None):
    """
    ========================================================================
    팀장의 Result.json을 분석하여 최종 리포트 생성
    ========================================================================

    입력:
        result_json: COCO JSON 딕셔너리 (또는 None)
        result_json_path: JSON 파일 경로 (문자열)
        path_manager: PathManager 또는 ResultPathManager 인스턴스

    출력:
        분석 결과 리스트 반환
    ========================================================================
    """

    # JSON 데이터 로드
    if result_json_path:
        with open(result_json_path, 'r', encoding='utf-8') as f:
            results = json.load(f)
    else:
        results = result_json

    print("\n" + "="*70)
    print("🔍 분석: 팀장의 Result.json 처리 중...")
    print("="*70)

    # 분석 결과 저장소
    analysis_results = []

    # 이미지별 분석
    for image_info in results.get('images', []):
        img_id = image_info['id']
        img_h = image_info['height']
        img_w = image_info['width']

        # 해당 이미지의 어노테이션 필터링
        image_annotations = [
            a for a in results['annotations']
            if a['image_id'] == img_id
        ]

        # 클래스별 통계
        class_stats = {}
        total_area = 0

        for ann in image_annotations:
            cat_id = ann['category_id']
            area = ann.get('area', 0)
            bbox = ann.get('bbox', [0, 0, 0, 0])

            if cat_id not in class_stats:
                class_stats[cat_id] = {
                    'count': 0,
                    'total_area': 0,
                    'avg_bbox': [0, 0, 0, 0],
                    'items': []
                }

            class_stats[cat_id]['count'] += 1
            class_stats[cat_id]['total_area'] += area
            class_stats[cat_id]['items'].append({
                'bbox': bbox,
                'area': area
            })
            total_area += area

        # 평균 바운딩박스 계산
        for cat_id in class_stats:
            items = class_stats[cat_id]['items']
            if items:
                avg_x = np.mean([item['bbox'][0] for item in items])
                avg_y = np.mean([item['bbox'][1] for item in items])
                avg_w = np.mean([item['bbox'][2] for item in items])
                avg_h = np.mean([item['bbox'][3] for item in items])
                class_stats[cat_id]['avg_bbox'] = [avg_x, avg_y, avg_w, avg_h]

        # 간단한 도로 혼잡도 계산 (어노테이션 수 기반)
        if len(image_annotations) > 10:
            congestion = "혼잡"
        elif len(image_annotations) > 5:
            congestion = "보통"
        else:
            congestion = "원활"

        # 안전 판정 (면적 기반 간단한 휴리스틱)
        safety_status = "SAFE"
        if total_area > (img_h * img_w * 0.5):  # 50% 이상 점유
            safety_status = "⚠️  WARNING"
        if total_area > (img_h * img_w * 0.7):  # 70% 이상 점유
            safety_status = "🚨 CRITICAL"

        # 결과 저장
        result = {
            'image_id': img_id,
            'file_name': image_info['file_name'],
            'image_size': (img_w, img_h),
            'annotation_count': len(image_annotations),
            'total_area': total_area,
            'congestion': congestion,
            'safety_status': safety_status,
            'class_stats': class_stats
        }

        analysis_results.append(result)

    # 종합 리포트
    print("\n📊 분석 결과 요약")
    print("-" * 70)

    if len(analysis_results) > 0:
        # 처음 5개 이미지만 상세 출력
        for idx, result in enumerate(analysis_results[:5], 1):
            print(f"\n[{idx}] {result['file_name']}")
            print(f"    ┌─ 이미지 크기: {result['image_size']}")
            print(f"    ├─ 어노테이션: {result['annotation_count']}개")
            print(f"    ├─ 점유 면적: {result['total_area']:.0f}px² "
                  f"({100*result['total_area']/(result['image_size'][0]*result['image_size'][1]):.1f}%)")
            print(f"    ├─ 혼잡도: {result['congestion']}")
            print(f"    └─ 상태: {result['safety_status']}")

            for cat_id, stats in result['class_stats'].items():
                cat_name = [c['name'] for c in results['categories'] if c['id'] == cat_id]
                cat_name = cat_name[0] if cat_name else f"Cat#{cat_id}"
                print(f"       • {cat_name}: {stats['count']}개, "
                      f"면적={stats['total_area']:.0f}, "
                      f"bbox={[round(x, 1) for x in stats['avg_bbox']]}")

        if len(analysis_results) > 5:
            print(f"\n   ... 이외 {len(analysis_results) - 5}개 이미지")

    # 전체 통계
    total_annotations = sum(r['annotation_count'] for r in analysis_results)
    avg_congestion = np.mean([
        {'혼잡': 3, '보통': 2, '원활': 1}.get(r['congestion'], 1)
        for r in analysis_results
    ]) if analysis_results else 0

    print("\n" + "="*70)
    print("📈 전체 통계")
    print("="*70)
    print(f"  • 처리된 이미지: {len(analysis_results)}개")
    print(f"  • 총 어노테이션: {total_annotations}개")
    print(f"  • 평균 혼잡도: {avg_congestion:.2f}/3 "
          f"({'혼잡' if avg_congestion > 2 else '보통' if avg_congestion > 1 else '원활'})")

    # CSV로 저장
    if path_manager:
        csv_path = path_manager.get_analysis_path()

        # DataFrame 구성
        df_data = []
        for result in analysis_results:
            df_data.append({
                'image_id': result['image_id'],
                'file_name': result['file_name'],
                'annotation_count': result['annotation_count'],
                'total_area': result['total_area'],
                'congestion': result['congestion'],
                'safety_status': result['safety_status']
            })

        df = pd.DataFrame(df_data)
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ 분석 결과를 CSV로 저장했습니다: {csv_path}")

    return analysis_results



# ============================================================================
# [박무림 파일] Section 5: 통합 실행 함수
# ============================================================================

In [ ]:
def run_integrated_system(model, test_loader, device, class_map, base_path=None):
    """
    ========================================================================
    팀장 코드 + 박무림 분석의 완전 통합 실행
    ========================================================================

    이 함수는 다음 순서로 실행됩니다:
    1. ResultPathManager 초기화
    2. save_predictions_to_coco() 실행 → Result.json 생성
    3. process_team_result_json() 실행 → 분석

    입력:
        model: 팀장의 세그멘테이션 모델
        test_loader: 테스트 데이터 로더
        device: 'cuda' 또는 'cpu'
        class_map: 클래스 매핑
        base_path: 저장 경로 (None이면 자동 설정)
    """

    print("\n" + "█"*70)
    print("  🚀 팀장 코드 + 박무림 분석 통합 시스템 시작")
    print("█"*70)

    # 1. 경로 관리자 초기화
    pm = ResultPathManager(base_path)

    # 2. 팀장 코드 실행: Result.json 생성
    print("\n[Step 1/2] 팀장 코드 실행")
    print("-" * 70)
    result_json = save_predictions_to_coco(
        model=model,
        test_loader=test_loader,
        device=device,
        class_map=class_map,
        path_manager=pm
    )

    # 3. 박무림 분석 실행
    print("\n[Step 2/2] 박무림 분석 실행")
    print("-" * 70)
    analysis_results = process_team_result_json(
        result_json=result_json,
        path_manager=pm
    )

    print("\n" + "█"*70)
    print("  ✅ 통합 시스템 완료")
    print("█"*70)

    return result_json, analysis_results



# ============================================================================
# 주요 기능 테스트 및 사용 예시
# ============================================================================

In [ ]:
if __name__ == "__main__":
    print("\n" + "="*70)
    print("✅ 통합 파일 로드 성공!")
    print("="*70)
    print("\n다음 함수들을 사용할 수 있습니다:")
    print("  1. run_integrated_system() - 전체 통합 실행")
    print("  2. save_predictions_to_coco() - 예측 결과 저장")
    print("  3. process_team_result_json() - 결과 분석")
    print("  4. MurimFinalIntegration - 지능형 분석 엔진")
    print("  5. PathManager - 박항아 경로 관리")
    print("  6. ResultPathManager - 박무림 결과 경로 관리")
    print("\n사용 예시:")
    print("  result_json, analysis = run_integrated_system(")
    print("      model=your_model,")
    print("      test_loader=test_loader,")
    print("      device=device,")
    print("      class_map=class_map")
    print("  )")





✅ 통합 파일 로드 성공!

다음 함수들을 사용할 수 있습니다:
  1. run_integrated_system() - 전체 통합 실행
  2. save_predictions_to_coco() - 예측 결과 저장
  3. process_team_result_json() - 결과 분석
  4. MurimFinalIntegration - 지능형 분석 엔진
  5. PathManager - 박항아 경로 관리
  6. ResultPathManager - 박무림 결과 경로 관리

사용 예시:
  result_json, analysis = run_integrated_system(
      model=your_model,
      test_loader=test_loader,
      device=device,
      class_map=class_map
  )
